In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
SWOB_DIR = ''
SWOB_FILE = ''

In [ ]:
import dask
import dask.bag as db
import dask.dataframe as dd
import dask.distributed
import os
import pathlib
import pandas as pd
import xml.dom.minidom
import seaborn as sns

import itertools

In [ ]:
import dask_jobqueue

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    cores=12,
    processes=6,
    memory='128G',
    env_extra=['source ~/.bash_profile','conda activate smc01'],
    name='smc01-dask',
    local_directory=DATA_DIR / 'dask',
    walltime='3:00:00'
)

In [ ]:
cluster.scale(jobs=2)  # Scale to two working nodes as configured.
client = dask.distributed.Client(cluster)

In [ ]:
client

# With dask bag

In [ ]:
def string_to_dict(obs_xml_string):
    obs_data = xml.dom.minidom.parseString(obs_xml_string)
    metadata = obs_data.getElementsByTagName('identification-elements')[0]

    metadata_dict = {}

    for element in metadata.childNodes:
        variable = element.attributes['name'].value
        value = element.attributes['value'].value
        metadata_dict[variable] = value
        
    obs_dict = {}

    elements = obs_data.getElementsByTagName('elements')[0]
    for element in elements.childNodes:
        variable = element.attributes['name'].value
        value = element.attributes['value'].value
        obs_dict[variable] = value
        
    return {**metadata_dict, **obs_dict}

In [ ]:
bag = db.read_text(SWOB_DIR + '20201125/*/*.xml')

In [ ]:
bag

In [ ]:
obs_dicts = bag.map(string_to_dict)

In [ ]:
obs_dicts

In [ ]:
sample = obs_dicts.random_sample(0.01).compute()

In [ ]:
sample_df = sample.to_dataframe()

In [ ]:
len(sample)

In [ ]:
for obs in sample:
    if 'air_temp' in obs:
        print(obs['air_temp'])

In [ ]:
obs_df = obs_dicts.to_dataframe()

In [ ]:
computed = obs_df.compute()

In [ ]:
computed.columns

In [ ]:
computed['avg_air_temp_pst10mts'] = pd.to_numeric(computed['avg_air_temp_pst10mts'])

In [ ]:
computed.groupby(by='wmo_synop_id').mean()

In [ ]:
computed[['date_tm', 'air_temp']]

In [ ]:
computed['air_temp'] = pd.to_numeric(computed['air_temp'])

In [ ]:
computed['air_temp'].plot()

In [ ]:
sns.lineplot(data=computed, x='date_tm', y='air_temp')